## Setup Connection to S3 Bucket: 

In [167]:
import boto3
import sagemaker
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [168]:
%%bash

aws s3 ls s3://${bucket}/

2023-03-04 00:23:32 sagemaker-studio-328876407652-1zzlgzyu0f7
2023-03-04 00:28:42 sagemaker-us-east-1-328876407652
2023-03-18 04:47:03 testfinalproject088


In [169]:
# To list the S3 Bucket content
!aws s3 ls s3://sagemaker-us-east-1-993410942383/content-project/airline_data/

                           PRE Airline_Customer_Service/
                           PRE WeatherUS_2016/
                           PRE airlineID_carrier_table/
                           PRE airline_flights_table/
                           PRE airport_codes_table/
                           PRE cancellation_table/
2023-03-12 22:40:29  941254313 WeatherEvents_Jan2016-Dec2021.csv


## Set S3 Source Location (Public S3 Bucket):

In [5]:
s3_public_path = "s3://sagemaker-us-east-1-993410942383/content-project/airline_data/"

In [6]:
%store s3_public_path

Stored 's3_public_path' (str)


## Set S3 Destination Location (Private S3 Bucket):

In [7]:
s3_private_path = "s3://testfinalproject088/content/".format(bucket)
print(s3_private_path)

s3://testfinalproject088/content/


In [8]:
%store s3_private_path

Stored 's3_private_path' (str)


## Create Athena Database & Establish Connection: 

In [6]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

In [170]:
# Import PyAthena
from pyathena import connect

In [171]:
ingest_create_athena_db_passed = False

In [172]:
%store -r ingest_create_athena_table_parquet_passed

no stored variable or alias ingest_create_athena_table_parquet_passed


In [173]:
# Create Athena Database
database_name = "testanalysis"

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [174]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [175]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS testanalysis


In [176]:
# Verify The Database Has Been Created Succesfully
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,testanalysis


## Create Tables For Athena Database: 

In [300]:
# Drop Table if Needed
drop_table_name = "Airline_Carrier_Id"

drop_table_query = f"DROP TABLE IF EXISTS {database_name}.{drop_table_name}"

df_show = pd.read_sql(drop_table_query, conn)
df_show.head()

""


### Cancellation Codes Table -

In [207]:
cancellation_s3 = 's3://sagemaker-us-east-1-993410942383/content-project/airline_data/cancellation_table'

In [208]:
table1_name = "cancellation_code"

statement1 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Code string,
         Description string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table1_name, cancellation_s3
)

In [209]:
pd.read_sql(statement1, conn)

""


In [210]:
# Verify the table was created by way of a query
query1 = "SELECT * FROM {}.{}".format(database_name, table1_name)
pd.read_sql(query1, conn)

,index,code,description
0,0,A,Carrier
1,1,B,Weather
2,2,C,National Air System
3,3,D,Security


### Airlline Customer Service Table -

In [296]:
customer_service_s3 = 's3://testfinalproject088/airline_data/customer_service/csv'

In [297]:
table2_name = "Airline_Customer_Service"

statement2 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Carrier string,
         Commitment string,
         Provided string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table2_name, customer_service_s3
)

In [298]:
pd.read_sql(statement2, conn)

""


In [299]:
query7 = "SELECT * FROM {}.{}".format(database_name, table2_name)
pd.read_sql(query2, conn)

,carrier,commitment,provided
0,Alaska,Rebook passenger on same airline at no additio...,1
1,Alaska,Rebook on partner airline or another airline w...,1
2,Alaska,Meal or meal cash/voucher when cancellation re...,1
3,Alaska,Complimentary hotel accommodations for any pas...,1
4,Alaska,Complimentary ground transportation to and fro...,1
5,Allegiant,Rebook passenger on same airline at no additio...,1
6,Allegiant,Rebook on partner airline or another airline w...,0
7,Allegiant,Meal or meal cash/voucher when cancellation re...,0
8,Allegiant,Complimentary hotel accommodations for any pas...,1
9,Allegiant,Complimentary ground transportation to and fro...,1


### Airline Carrier ID Table - 

In [301]:
carrier_id_s3 = 's3://testfinalproject088/airline_data/carrier id/csv/'

In [302]:
table3_name = "Airline_Carrier_Id"

statement3 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Code string,
         Description string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table3_name, carrier_id_s3
)

In [303]:
pd.read_sql(statement3, conn)

""


In [304]:
query3 = "SELECT * FROM {}.{}".format(database_name, table3_name)
pd.read_sql(query3, conn)

,index,code,description
0,0,19031,Mackey International Inc.: MAC
1,1,19032,Munz Northern Airlines Inc.: XY
2,2,19033,Cochise Airlines Inc.: COC
3,3,19034,Golden Gate Airlines Inc.: GSA
4,4,19035,Aeromech Inc.: RZZ
...,...,...,...
1605,1605,21694,Silk Way West Airlines: 7L
1606,1606,21697,Orenburg Airlines: R2
1607,1607,21702,WestJet Encore Ltd.: WR
1608,1608,21707,Hong Kong Airlines Limited: HX


### Airport Codes Table -

In [211]:
airport_codes_s3 = 's3://sagemaker-us-east-1-993410942383/content-project/airline_data/airport_codes_table'

In [267]:
table4_name = "airline_codes"

statement4 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Code string,
         Description string
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
 WITH SERDEPROPERTIES ("separatorChar" = ",") 
 LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table4_name, airport_codes_s3
)

In [267]:
table4_name = "airline_codes"

statement4 = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Index string,
         Code string,
         Description string
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
 WITH SERDEPROPERTIES ("separatorChar" = ",") 
 LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table4_name, airport_codes_s3
)

In [268]:
pd.read_sql(statement4, conn)

""


In [269]:
query4 = "SELECT * FROM {}.{}".format(database_name, table4_name)
pd.read_sql(query4, conn)

,index,code,description
0,0,01A,"""Afognak Lake"
1,1,03A,"""Granite Mountain"
2,2,04A,"""Lik"
3,3,05A,"""Little Squaw"
4,4,06A,"""Kizhuyak"
...,...,...,...
6380,6380,ZXZ,"""Waterville"
6381,6381,ZYL,"""Sylhet"
6382,6382,ZZU,"""Mzuzu"
6383,6383,ZZV,"""Zanesville"


## Visualize Dataset: 

In [16]:
df = pd.read_csv("s3://sagemaker-us-east-1-993410942383/content-project/airline_data/airport_codes_table.csv")
df.head()

,Unnamed: 0,Code,Description
0,0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,2,04A,"Lik, AK: Lik Mining Camp"
3,3,05A,"Little Squaw, AK: Little Squaw Airport"
4,4,06A,"Kizhuyak, AK: Kizhuyak Bay"


In [20]:
df2 = pd.read_csv("s3://sagemaker-us-east-1-993410942383/content-project/airline_data/cancellation_table/CancellationCode_table.csv")
df2.head()

,Unnamed: 0,Code,Description
0,0,A,Carrier
1,1,B,Weather
2,2,C,National Air System
3,3,D,Security


## Possible questions to explore includes:

1. What is the most common reason for flights being cancelled? 

### 1. What is the most common reason for flights being cancelled? 

In [ ]:
# SQL statement
statement = """
SELECT product_category, AVG(star_rating) AS avg_star_rating
FROM {}.{} 
GROUP BY product_category 
ORDER BY avg_star_rating DESC
""".format(
    database_name, table_name
)

print(statement)